# Определение стоимости автомобилей

In [20]:
# from google.colab import drive
# drive.mount('/content/drive')

# Описание проекта
Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение, чтобы привлечь новых клиентов. В нём можно будет узнать рыночную стоимость своего автомобиля.
Постройте модель, которая умеет её определять. В вашем распоряжении данные о технических характеристиках, комплектации и ценах других автомобилей.
Критерии, которые важны заказчику:
    качество предсказания;
    время обучения модели;
    время предсказания модели.

In [21]:
!pip install lightgbm
!pip install catboost
!pip install category_encoders
!pip install fast_ml

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder
from category_encoders import MEstimateEncoder
from fast_ml.model_development import train_valid_test_split

In [23]:
try:
    df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/autos.csv')
except:
    df = pd.read_csv('/datasets/autos.csv')

display(df.head())
df.info()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

# Изучите данные.
Заполните пропущенные значения и обработайте аномалии в столбцах. Если среди признаков имеются неинформативные, удалите их.

In [24]:
# приведем названия столбцов
df.columns = df.columns.str.replace(r"([A-Z])", r" \1").str.lower().str.replace(' ', '_').str[1:]

In [25]:
# удалим столбцы, которые не помогут в обучении модели
df = df.drop(['date_crawled', 'registration_month', 'date_created', 'number_of_pictures', 'postal_code', 'last_seen'], axis=1)

In [26]:
df.isna().sum()

price                    0
vehicle_type         37490
registration_year        0
gearbox              19833
power                    0
model                19705
kilometer                0
fuel_type            32895
brand                    0
repaired             71154
dtype: int64

In [27]:
display(df['vehicle_type'].value_counts())
# заменим NaN на other
df['vehicle_type'] = df['vehicle_type'].fillna('other')

sedan          91457
small          79831
wagon          65166
bus            28775
convertible    20203
coupe          16163
suv            11996
other           3288
Name: vehicle_type, dtype: int64

In [28]:
display(df['gearbox'].value_counts())
df['gearbox'].fillna('unknown', inplace=True)

manual    268251
auto       66285
Name: gearbox, dtype: int64

In [29]:
display(df['model'].value_counts())
df['model'].fillna('other', inplace=True)

golf                  29232
other                 24421
3er                   19761
polo                  13066
corsa                 12570
                      ...  
i3                        8
serie_3                   4
rangerover                4
range_rover_evoque        2
serie_1                   2
Name: model, Length: 250, dtype: int64

In [30]:
display(df['fuel_type'].value_counts())
df['fuel_type'].fillna('other', inplace=True)
# gasoline и petrol - одно и то же, следует обьединить
df['fuel_type'] = df['fuel_type'].replace('gasoline','petrol')

petrol      216352
gasoline     98720
lpg           5310
cng            565
hybrid         233
other          204
electric        90
Name: fuel_type, dtype: int64

In [31]:
display(df['repaired'].value_counts())
df['repaired'].fillna('other', inplace=True)
# перепроверим пропущеные значения
df.isna().sum()

no     247161
yes     36054
Name: repaired, dtype: int64

price                0
vehicle_type         0
registration_year    0
gearbox              0
power                0
model                0
kilometer            0
fuel_type            0
brand                0
repaired             0
dtype: int64

In [32]:
# Проверяем на наличие дубликатов
print('Дубликатов:', df.duplicated().sum())
df = df.drop_duplicates().reset_index(drop=True)
print('Дубликатов после удаления:', df.duplicated().sum())

Дубликатов: 46042
Дубликатов после удаления: 0


In [33]:
# Ищем аномальные значения
display(df.describe())
# Замечаем аномалии: год регистрации 9999, количесто лошадиных сил 0 и 20000

df = df[(df['registration_year'] > 1930) & (df['registration_year'] < 2016)]
#Функция для обработки аномалий
def balance(df, column):
    q25 = df[column].quantile(0.25)
    q75 = df[column].quantile(0.75)
    iqr = q75 - q25
    lower = q25 - 1.5 * iqr
    upper = q75 + 1.5 * iqr
    del_index = df[(df[column] < lower) | (df[column] > upper)].index
    print(f"Количество строк, выбранных для удаления {column}: {len(del_index)}")
    return del_index

col = ["price", "power"]
count = 0
for column in col:
    to_del = balance(df, column)
    count += len(to_del)
df = df.drop(to_del, axis=0)
print("Было удалено всего строк:", count)
display(df.describe())


,price,registration_year,power,kilometer
count,308327.000000,308327.000000,308327.000000,308327.000000
mean,4492.320854,2004.367597,111.044164,127167.325599
std,4568.472330,92.690667,201.268827,38557.473682
min,0.000000,1000.000000,0.000000,5000.000000
25%,1100.000000,1999.000000,69.000000,125000.000000
50%,2800.000000,2003.000000,105.000000,150000.000000
75%,6500.000000,2008.000000,143.000000,150000.000000
max,20000.000000,9999.000000,20000.000000,150000.000000


Количество строк, выбранных для удаления price: 13066
Количество строк, выбранных для удаления power: 6489
Было удалено всего строк: 19555


,price,registration_year,power,kilometer
count,279130.000000,279130.000000,279130.000000,279130.000000
mean,4492.033178,2002.101673,104.896919,126689.033784
std,4502.624517,6.757793,56.448676,38648.097784
min,0.000000,1931.000000,0.000000,5000.000000
25%,1100.000000,1999.000000,70.000000,125000.000000
50%,2850.000000,2002.000000,105.000000,150000.000000
75%,6500.000000,2007.000000,140.000000,150000.000000
max,20000.000000,2015.000000,251.000000,150000.000000


* После подготовки даных в общей сложности было удалено 53 тыс. строк. Среди них были как аномальные значения, так и дубликаты.
* Исправлены наименования столбцов для удобочитаемости, а так же обединены некоторые группые категориальных признаков, которые дублируют друг друга.
* Датасет очищен от отсутствующих значений.
* Категориальные признаки переведены в количественные в отдельной копии датасете

# Подготовьте выборки для обучения моделей.


In [38]:
features_train, target_train, features_valid, target_valid, features_test, target_test = train_valid_test_split(df, target='price', train_size=0.6, valid_size=0.2, test_size=0.2, random_state=42)

def encode(data, target):
    # Подготавливаем инструменты для масштабирования и кодирования
    scaler = StandardScaler()
    encoder_ohe = OneHotEncoder(drop='first', sparse=False)
    encoder_m_estimate = MEstimateEncoder()

    # Отделяем количественные и категориальные признаки
    numerical_features = data.select_dtypes(include=[np.number]).columns.tolist()
    categorical_features = data.select_dtypes(include=['object']).columns.tolist()

    # Не забываем про размерность
    categorical_features.remove('brand')
    categorical_features.remove('model')

    # Масштабируем количественные признаки
    scaler.fit(features_train[numerical_features])
    scaled_numerical = scaler.transform(data[numerical_features])

    # Применяем OHE к категориальным признакам (кроме 'brand' и 'model')
    encoder_ohe.fit(features_train[categorical_features])
    ohe_features = encoder_ohe.transform(data[categorical_features])

    # MEstimate для Brand и Model
    encoder_m_estimate.fit(features_train[['brand', 'model']], target_train)
    encoded_razmernost = encoder_m_estimate.transform(data[['brand', 'model']])
    
    # Объединяем масштабированные и закодированные признаки, не забыв сбросит индексы
    features_final = pd.concat([
    pd.DataFrame(scaled_numerical, columns=numerical_features),
    pd.DataFrame(ohe_features, columns=encoder_ohe.get_feature_names(input_features=categorical_features)),
    encoded_razmernost.reset_index(drop=True)
], axis=1)

    return features_final

# Применяем функцию к разным выборкам
features_train_num = encode(features_train, target_train)
features_valid_num = encode(features_valid, target_valid)
features_test_num = encode(features_test, target_test)

print('Размер тренировочной выборки:', features_train_num.shape)
print('Размер валидационной выборки:', features_valid_num.shape)
print('Размер тестовой выборки:', features_test_num.shape)


Размер тренировочной выборки: (167478, 21)
Размер валидационной выборки: (55826, 21)
Размер тестовой выборки: (55826, 21)


## Кодируем выборки

In [ ]:
# Отдельно создадим выборку без предварительного перевода категориальных признаком и без масштабирования используя fast_ml
features_train, target_train, features_valid, target_valid, features_test, target_test = train_valid_test_split(df, target = 'price', train_size=0.6, valid_size=0.2, test_size=0.2)
print('Размер тренировочной выборки:', features_train.shape)
print('Размер валидационной выборки:', features_valid.shape)
print('Размер тестовой выборки:', features_test.shape)

# Обучите разные модели
одна из которых — LightGBM, как минимум одна — не бустинг. Для каждой модели попробуйте разные гиперпараметры.

In [ ]:
# catboost

model = CatBoostRegressor(loss_function="RMSE", iterations=10)
cat_features = ['vehicle_type', 'gearbox', 'model',
                'fuel_type', 'brand', 'repaired']
parameters_cat = {'depth':[1,10, 2],
                  'learning_rate':(0.1,0.4,0.7,0.9)}

catboost_grid = model.grid_search(parameters_cat,
            Pool(features_valid[features_valid.columns], target_valid, cat_features=cat_features),
            cv=3,
            verbose=False,
            plot=False)

In [ ]:
catboost_grid["params"]

In [ ]:
%%time
catboost = CatBoostRegressor(depth=catboost_grid["params"]["depth"],
                              learning_rate=catboost_grid["params"]['learning_rate'],
                              loss_function='RMSE',verbose=100)
catboost.fit(Pool(features_train, target_train,
                   cat_features=cat_features))

In [ ]:
%%time
cat_predict = catboost.predict(features_test)
print('Время предсказания CatBoost: ', cat_predict)

In [ ]:
print("CatBoost RMSE:", mean_squared_error(target_test, cat_predict) ** 0.5)

In [ ]:
# RandomForest

RF = RandomForestRegressor(criterion = "friedman_mse",random_state=42,n_jobs=-1)

# Подбор гиперпараметров для RandomForest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3,5,7],
}

grid_search = GridSearchCV(estimator=RF, param_grid=param_grid, scoring='neg_root_mean_squared_error',
                           cv=3, verbose=1, n_jobs=-1)

grid_search.fit(features_valid_num, target_valid_num)

In [ ]:
# Лучшие параметры
best_est = grid_search.best_params_['n_estimators']
best_depth = grid_search.best_params_['max_depth']
print('Лучшее est', best_est)
print('Лучшая глубина', best_depth)
print('Лучшая RMSE RandomForest на валидационной выборке : ', grid_search.best_score_*-1)

In [ ]:
%%time
# Обучаем на тренировочных данных и лучших параметрах
RF = RandomForestRegressor(random_state=42, n_estimators=best_est, max_depth=best_depth)
RF.fit(features_train_num, target_train_num)

In [ ]:
%%time
# Получите предсказания на тестовых данных и оцените RMSE
predictions = RF.predict(features_test_num)
rmse = mean_squared_error(target_test_num, predictions)**0.5

# Выведите результаты
print('Лучшее количество деревьев:', best_est)
print('Лучшая глубина:', best_depth)
print('RMSE для случайного леса с лучшими гиперпараметрами на тестовой выборке:', rmse)

In [ ]:
# построим модель без подбора гиперпараметров
model = lgb.LGBMRegressor(random_state=42)
model.fit(features_train_num, target_train_num)
predictions_train = model.predict(features_train_num)
rmse = mean_squared_error(target_train_num, predictions_train)**0.5
print('RMSE lgbm на тренировочный выборке без предустановленных параметров:', rmse)


In [ ]:
model = lgb.LGBMRegressor(random_state=42)
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}
tuning_model = GridSearchCV(estimator=model,
                            param_grid=param_grid,
                            scoring='neg_root_mean_squared_error',
                            cv=5,
                            verbose=1,
                            n_jobs=-1)
tuning_model.fit(features_valid_num, target_valid_num)

In [ ]:
print('Лучшие параметры LGBM:', tuning_model.best_params_)
print('RMSE LGBM на валидационной выборке : ', tuning_model.best_score_*-1)

In [ ]:
%%time
model = lgb.LGBMRegressor(random_state=42, n_estimators=300, learning_rate=0.1, max_depth=7)
model.fit(features_train_num, target_train_num)

In [ ]:
%%time
lgb_predict = model.predict(features_test_num)
print('Время предсказания LGBM: ', lgb_predict)

In [ ]:
print("LightGBM RMSE на тестовой выборке:",(mean_squared_error(target_test_num,lgb_predict)**0.5))

Выводы:

> CatBoost:
Время обучения: 9min 12s
Время предсказания: 1.64 s
RMSE: 1715.281


---


> RandomForest:
Время обучения: 57.3 s
Время предсказания: 687 ms
RMSE: 2117.214


---


>LGBM:
Время обучения: 2.92 s
Время предсказания: 765 ms
RMSE : 1786.6348

> Модель LighGBM показала преимущество перед другими моделями по скорости обучения и времени предсказания, незначительно уступив Catboost по метрике RMSE.

> Сatboost в свою очередь показал худшие временные показатели, что может сказаться на пользовательском опыте приложения.

# Общий вывод:


> В ходе предобработки исходного датасета в общей сложности было удалено 53 тыс. строк. Среди них были как аномальные значения, так и дубликаты. Исправлены наименования столбцов для удобочитаемости, а так же обединены некоторые группые категориальных признаков, которые дублируют друг друга. Датасет очищен от отсутствующих значений.
Категориальные признаки переведены в количественные в отдельной копии датасете не забывая о проклятии размерности.

> В ходе подготовки выборок для оубчения было решено поделить их на 3 выборки в пропорции 3-1-1: для подбора обучения(тренировочная), для подбора гиперпараметров(валидационная), для финального тестирования моделия(тестовая). Отдельно для каждой выборки проведено масштабирование признаков и кодирование.

> В ходе работы с 3-мя моделями МО, наилучший показатель RMSE продемонстрировала модель Catboost - 1715.281, но по временным параметрам уступив двум другим моделям.

> Наилучшие скоростные характеристики имеет модель LightGBM, а RMSE назначиельно хуже Catboost - 1786.634.

Исходя из требований заказчика по скорости обучения, придсказания и качества модели, рекомендуем модель LightGBM, поскольку она имеет самые оптимальные характеристики.